In [1]:
!pip install transformers
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install nltk
!pip install torch
!pip install bert_score
!pip install torchtext
!pip install language-tool-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [2]:
import pandas as pd
import numpy as np
from datasets import load_dataset
import tensorflow as tf
import torch

In [3]:
# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [4]:
# Define dataset path
dataset_path = "cfilt/iitb-english-hindi"
train_dataset = load_dataset(dataset_path, split="train[:1000]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/1659083 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/520 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2507 [00:00<?, ? examples/s]

In [5]:
english_sentences = [item['en'] for item in train_dataset['translation']]
hindi_sentences = [item['hi'] for item in train_dataset['translation']]

In [6]:
from transformers import pipeline

pipe = pipeline("translation", model="facebook/nllb-200-distilled-600M")

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M").to(device)
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang="hin_Deva", tgt_lang='eng_Latn', max_length = 400, device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/846 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.3M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
translatedHinEng = []
for (index, (eng, hin)) in enumerate(zip(english_sentences, hindi_sentences)):
    print(index+1)
    print("Hindi:", hin)
    prediction = translator(hin)
    translation = prediction[0]['translation_text']
    print("Translated: ", translation)
    print("Original: ", eng)
    print("------------------------------")
    translatedHinEng.append(translation)

comparison = pd.DataFrame({'Original': english_sentences, 'Translated': translatedHinEng})
comparison.to_csv('/content/IITB_Translated_EngHin.csv', index=False)

1
Hindi: अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें
Translated:  Give your application the benefit of accessibility exercise
Original:  Give your application an accessibility workout
------------------------------
2
Hindi: एक्सेर्साइसर पहुंचनीयता अन्वेषक
Translated:  Exerciser Accessibility Investigator
Original:  Accerciser Accessibility Explorer
------------------------------
3
Hindi: निचले पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug-in template for the lower panel
Original:  The default plugin layout for the bottom panel
------------------------------
4
Hindi: ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका
Translated:  Default plug-in template for the upper panel
Original:  The default plugin layout for the top panel
------------------------------
5
Hindi: उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है
Translated:  List of plugins that are disabled by default
Original:  A list of plugins that are disabled by default
------------------------------
6
Hindi:

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Translated:  The colour and opacity of the outline highlighted.
Original:  The color and opacity of the highlight border.
------------------------------
10
Hindi: भराई के रंग को हाइलाइट करें
Translated:  Highlight the color of the filling
Original:  Highlight fill color
------------------------------
11
Hindi: हाइलाइट किया गया भराई का रंग और पारदर्शिता। 
Translated:  Highlighted color and transparency of the filler.
Original:  The color and opacity of the highlight fill.
------------------------------
12
Hindi: एपीआई विचरक
Translated:  API diffuser
Original:  API Browser
------------------------------
13
Hindi: इस समय जिसे प्राप्त किया गया हो, उसकी विभिन्न विधियों (मेथड) में विचरण करें
Translated:  Diverge between the different methods that have been obtained at this time
Original:  Browse the various methods of the current accessible
------------------------------
14
Hindi: निजी गुणों को छिपाएं
Translated:  Hide private qualities
Original:  Hide private attributes
--------------------

In [8]:
# BLEU Score
import nltk
from nltk.translate.bleu_score import corpus_bleu
references = [[eng] for eng in english_sentences]  # Create reference list of lists
candidates = translatedHinEng  # Model translations
bleu_score = corpus_bleu(references, candidates)
print(f"BLEU score: {bleu_score:.4f}")

BLEU score: 0.5664


In [9]:
# STS Score

!pip install sentence-transformers

# STS Score using SentenceTransformer
from sentence_transformers import SentenceTransformer, util
import torch

# Load the pre-trained model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# Encode the original Hindi sentences and the translated sentences
embeddings1 = model.encode(english_sentences, convert_to_tensor=True, device=model.device)
embeddings2 = model.encode(translatedHinEng, convert_to_tensor=True, device=model.device)

# Compute cosine similarities between the embeddings
cosine_scores = util.cos_sim(embeddings1, embeddings2)

# Calculate the average STS score
sts_score = cosine_scores.diag().mean().item()
print(f"STS score: {sts_score:.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 5.6 MB/s eta 0:00:00


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

STS score: 0.7955


In [10]:
# CHRF Score
!pip install sacrebleu
from sacrebleu.metrics import CHRF

# Initialize CHRF metric
chrf = CHRF(word_order=1)

# Calculate CHRF score
chrf_score = chrf.corpus_score(translatedHinEng, [english_sentences]).score
print(f"CHRF score: {chrf_score:.4f}")


CHRF score: 55.4770


In [14]:
# METEOR Score
import nltk
from nltk.translate.meteor_score import meteor_score

# Download required NLTK data (if not already downloaded)
nltk.download('punkt')
nltk.download('wordnet')

# Tokenize the sentences if they're not already tokenized
english_sentences_tokenized = [nltk.word_tokenize(sent) for sent in english_sentences]
translatedHinEng_tokenized = [nltk.word_tokenize(sent) for sent in translatedHinEng]

# Calculate METEOR score
meteor_scores = [meteor_score([ref], hyp) for ref, hyp in zip(english_sentences_tokenized, translatedHinEng_tokenized)]
average_meteor_score = sum(meteor_scores) / len(meteor_scores)

print(f"METEOR score: {average_meteor_score:.4f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


METEOR score: 0.4665


In [15]:
import nltk
from nltk.lm import KneserNeyInterpolated
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.util import ngrams
import numpy as np

# Download necessary NLTK data
nltk.download('punkt')

def train_model(sentences, n=2):
    tokenized = [['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>'] for sent in sentences]
    train_data, padded_vocab = padded_everygram_pipeline(n, tokenized)
    model = KneserNeyInterpolated(n)
    model.fit(train_data, padded_vocab)
    return model

def calculate_perplexity(model, sentences, n=2):
    perplexities = []
    for sent in sentences:
        tokens = ['<s>'] + list(map(str.lower, nltk.word_tokenize(sent))) + ['</s>']
        test_data = list(ngrams(tokens, n))
        try:
            pp = model.perplexity(test_data)
            perplexities.append(pp)
        except ValueError:
            pass  # Skip sentences that cause errors
    return np.mean(perplexities) if perplexities else float('inf')

# Train models
english_model = train_model(english_sentences)
hindi_model = train_model(translatedHinEng)

# Calculate perplexities
english_perplexity = calculate_perplexity(english_model, english_sentences)
hindi_perplexity = calculate_perplexity(hindi_model, translatedHinEng)

print(f"English Perplexity: {english_perplexity:.4f}")
print(f"Translated hindi Perplexity: {hindi_perplexity:.4f}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


English Perplexity: 11.4865
Translated hindi Perplexity: 11.9911
